### Reqs

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
with open('/content/drive/MyDrive/Colab Notebooks/api_keys/apikeys.txt') as file:
    hftoken = file.readlines()
    hftoken = hftoken[0][4:]

In [ ]:
#@title logging to huggingface
import shutil,os,re,glob,sys,time,cv2
HF_TOKEN =
from IPython.display import clear_output
!pip install -q huggingface_hub
from huggingface_hub import login
login(token=
      hftoken
      )
!pip install huggingface-cli
clear_output()

In [ ]:
from IPython.display import clear_output
import numpy as np

%pip install num2words --quiet
%pip install av decord --quiet
%pip install kernels --quiet
!pip install -q transformers==4.54.0, trl==0.12.2, datasets==3.2.0, bitsandbytes==0.46.0, peft==0.14.0, accelerate==1.2
import num2words
clear_output()

### Dataset

In [ ]:
!pip install datasets
from datasets import load_dataset
repo_id = "AnasKAN/Muraqeb_clean_dataset"
ds = load_dataset(repo_id)
train_ds = ds["train"]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/982 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/13.3M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/6.45M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/50238 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/23940 [00:00<?, ? examples/s]

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'images', 'text prompt', 'video link', 'visual quality', 'temporal consistency', 'dynamic degree', 'text-to-video alignment', 'factual consistency', 'conversations', 'question', 'qa_type', 'source_json', 'sample_uid'],
        num_rows: 50238
    })
    test: Dataset({
        features: ['id', 'images', 'text prompt', 'video link', 'visual quality', 'temporal consistency', 'dynamic degree', 'text-to-video alignment', 'factual consistency', 'conversations', 'question', 'qa_type', 'source_json', 'sample_uid'],
        num_rows: 23940
    })
})

### Loading the model

In [ ]:
#@title flash attention dependency
from IPython.display import clear_output
!pip install -q flash-attn --no-build-isolation

'''
if you don't install it this error will appear
ImportError: FlashAttention2 has been toggled on, but it cannot be used due to the following error: the package flash_attn seems to be not installed. Please refer to the documentation of https://huggingface.co/docs/transformers/perf_infer_gpu_one#flashattention-2 to install Flash Attention 2.

and if you restart the session sometimes it will produce an error this one:
ImportError: /usr/local/lib/python3.11/dist-packages/flash_attn_2_cuda.cpython-311-x86_64-linux-gnu.so: undefined symbol: _ZN3c105ErrorC2ENS_14SourceLocationENSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE
'''
clear_output()

In [ ]:
from transformers import AutoProcessor, AutoModelForImageTextToText
import torch

model_path = "AnasKAN/SmolVLM2-500M-Video-Instruct-video-feedback" # Full FT model
model_path = "AnasKAN/SmolVLM2-500M-Video-Instruct-qlora-tuned" # QLoRA model
model_path = "HuggingFaceTB/SmolVLM2-500M-Video-Instruct" # Base model

processor = AutoProcessor.from_pretrained('HuggingFaceTB/SmolVLM2-500M-Video-Instruct')
model = AutoModelForImageTextToText.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,
    _attn_implementation="flash_attention_2"
).to("cuda")

### Inference

In [ ]:
#@title cleaning function
target_size   = (640, 360)   # (width, height)
target_frames = 128
output_fps    = 24
fourcc        = cv2.VideoWriter_fourcc(*"XVID")  # saves .avi

def sample_indices(n_frames_src, n_target):
    """Select evenly spaced frame indices."""
    if n_frames_src <= 0:
        return []
    idxs = np.linspace(0, n_frames_src - 1, min(n_target, n_frames_src), dtype=int)
    return idxs.tolist()

def clean_single_video(video_path, save_dir, overwrite=False):
    """
    Clean one video and save as {basename}_clean.avi in save_dir.

    Args:
        video_path (str): path to raw video
        save_dir (str): directory to save cleaned video
        overwrite (bool): if False and cleaned file exists, skip

    Returns:
        str or None: path to cleaned video if success, None otherwise
    """
    os.makedirs(save_dir, exist_ok=True)
    base = os.path.splitext(os.path.basename(video_path))[0]
    dst_path = os.path.join(save_dir, f"{base}_clean.avi")

    if os.path.exists(dst_path) and not overwrite:
        print(f"[SKIP] {dst_path} already exists.")
        return dst_path

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"[ERROR] Cannot open {video_path}")
        return None

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    idxs = sample_indices(total_frames, target_frames)

    frames = []
    grabbed = 0
    cur = 0
    next_targets = set(idxs)

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if cur in next_targets:
            frame = cv2.resize(frame, target_size, interpolation=cv2.INTER_AREA)
            frames.append(frame)
            grabbed += 1
            if grabbed == len(idxs):
                break
        cur += 1
    cap.release()

    if len(frames) == 0:
        print(f"[ERROR] No frames extracted from {video_path}")
        return None

    # pad to target_frames if needed
    while len(frames) < target_frames:
        frames.append(frames[-1])

    out = cv2.VideoWriter(dst_path, fourcc, output_fps, target_size)
    for f in frames:
        out.write(f)
    out.release()

    print(f"[OK] {video_path} -> {dst_path} ({len(frames)} frames @ {output_fps} FPS)")
    return dst_path


In [ ]:
video_path = "/content/data/videos/Train/D001_01.avi"
save_dir   = "/content/clean_videos"

cleaned_path = clean_single_video(video_path, save_dir)
print("Cleaned video:", cleaned_path)

In [ ]:
prompt = "Describe this video in detail"

video_path = "/content/clean_videos/D001_01_clean.avi"

messages = [
    {
        "role": "user",
        "content": [
            {"type": "video", "path": video_path},
            {"type": "text", "text": prompt}
        ]
    },
]

inputs = processor.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt",
    video_load_backend='decord'
).to(model.device, dtype=torch.bfloat16)

start = time.time()
generated_ids = model.generate(
    **inputs,
    do_sample=False,
    max_new_tokens=256
)
end = time.time()

elapsed = end - start

generated_texts = processor.batch_decode(
    generated_ids,
    skip_special_tokens=True,
)
num_tokens = generated_ids.shape[-1] - inputs["input_ids"].shape[-1]
tps = num_tokens / elapsed if elapsed > 0 else float("inf")

print("="*40)
print(f"Generation time : {elapsed:.2f} sec")
print(f"New tokens      : {num_tokens}")
print(f"Speed           : {tps:.2f} tokens/sec")
print("="*40)
print("Generated text:")
print(generated_texts[0])